In [1]:
import time
import pandas as pd
import numpy as np
from langchain_community.llms import ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

In [11]:
data = pd.read_csv("../data/interim/merged_scrapped_n_price.csv")

In [3]:
prompt_template = """
    You are an expert financial analyst specialized in currency markets, particularly in evaluating
    the impact of news articles on the EURUSD currency pair. Your task is to read the following news
    article and analyze its sentiment to determine its bias toward either the Euro (EUR) or
    the US Dollar (USD).

    Provide a single floating-point value between 0 and 1 that represents this bias:
    - 0.0000: Extremely bullish for the Euro (EUR)
    - 1.0000: Extremely bullish for the US Dollar (USD)
    - 0.5000: Neutral, with no bias toward either currency

    **Instructions:**
    1. Assess the content of the article for references to economic indicators, policies, or events that could impact the EURUSD currency pair.
    2. Weigh any implications of the news against typical market reactions to similar news for the EUR and USD.
    3. Avoid explanations or reasoning in your response; provide only the float value to four decimal places.

    Example Input:\n
    Nonfarm business sector labor productivity increased 3.4 percent in the first quarter of 2019,
    the U.S. Bureau of Labor Statistics reported today, as output increased 3.9 percent and hours
    worked increased 0.5 percent. (All quarterly percent changes in this release are seasonally
    adjusted annual rates.) From the first quarter of 2018 to the first quarter of 2019, productivity
    increased 2.4 percent, reflecting a 3.9- percent increase in output and a 1.5-percent increase in
    hours worked. (See table A1.) The four-quarter increase in productivity is the largest since a
    2.7-percent gain in the third quarter of 2010. Labor ... (full story)


    Example Output:\n
    0.9000

    \n\n

    Article:\n {article}?\n

    Answer:
"""

In [4]:
def get_response_chain(prompt_template: str, model_name: str= "mistral-nemo"):
    """
    Set up model parameters and prompt template
    """

    model = ollama.Ollama(
        model = model_name,
        temperature = 0.0,
        verbose=True
    )

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["article"],
    )

    chain = prompt | model | StrOutputParser()
    return chain

In [ ]:
def llm_sentiment(prompt_template: str, article: str) -> float:

    chain = get_response_chain(prompt_template=prompt_template)

    start = time.time()
    response = chain.invoke(
        {
            "article": article,
        },
        return_only_outputs=True
    )
    llm_response_load_time = time.time() - start
    print(f"LLM response load time: {llm_response_load_time:.3f}s")

    return np.float16(response)

In [9]:
data.columns

Index(['datetime', 'impact', 'link_text'], dtype='object')

In [10]:
text = data.loc[100, "link_text"]
print(text)
llm_sentiment(prompt_template=prompt_template, article=text)

The euro zone's private sector started 2014 in much better shape than expected, with stronger growth across the region marred only by a continued downturn in France, surveys showed on Thursday. Markit's Flash Eurozone Composite Purchasing Managers' Index (PMI), which gauges business activity across thousands of companies and is seen as a good guide to economic health, jumped to 53.2 in January from 52.1 last month. That was well above the 50 mark that denotes growth and was its highest reading since mid-2011, beating all forecasts in a Reuters poll of 25 economists. "Hopefully the upturn we are seeing in Germany and ... (full story)
 The euro area private sector economy grew for a seventh consecutive month in January, according to the flash Markit Eurozone PMI®, with the rate of growth accelerating to the fastest since June 2011. The headline PMI (which tracks output across both manufacturing and services) rose from 52.1 in December to 53.2. Growth picked up in Germany and the rate of 

'0.2500'

In [2]:
parsed_data = pd.read_csv("../data/interim/parsed_scraped_data.csv")

In [17]:
parsed_data.head(100)

,time,open,high,low,close,tick_volume,spread,real_volume,impact,link_text,sentiment_score
0,2014-01-02 08:15:00,1.37422,1.37492,1.37421,1.37479,415,3,0,Medium Impact Expected,The Spanish manufacturing PMI signalled an imp...,0.65
1,2014-01-02 08:30:00,1.37484,1.37485,1.37347,1.37355,629,1,0,Medium Impact Expected,The Spanish manufacturing PMI signalled an imp...,0.65
2,2014-01-02 08:45:00,1.37354,1.37357,1.37254,1.37314,650,1,0,Medium Impact Expected,The Spanish manufacturing PMI signalled an imp...,0.65
3,2014-01-02 09:00:00,1.37320,1.37325,1.37159,1.37210,833,1,0,Low Impact Expected,The Spanish manufacturing PMI signalled an imp...,0.65
4,2014-01-02 09:15:00,1.37207,1.37252,1.37162,1.37184,806,1,0,Low Impact Expected,The Spanish manufacturing PMI signalled an imp...,0.65
...,...,...,...,...,...,...,...,...,...,...,...
95,2014-01-03 08:00:00,1.36517,1.36579,1.36484,1.36544,383,2,0,High Impact Expected,Construction spending increased 1 percent to a...,0.65
96,2014-01-03 08:15:00,1.36544,1.36616,1.36536,1.36607,400,1,0,High Impact Expected,Construction spending increased 1 percent to a...,0.65
97,2014-01-03 08:30:00,1.36606,1.36612,1.36565,1.36579,429,1,0,High Impact Expected,Construction spending increased 1 percent to a...,0.65
98,2014-01-03 08:45:00,1.36579,1.36582,1.36450,1.36506,519,1,0,High Impact Expected,Construction spending increased 1 percent to a...,0.65


In [22]:
"""
BREAKING
 The Labor Department reports on the number of people who applied for U.S. unemployment benefits last week. The report will be released at 8:30 a.m. Eastern Thursday. SMALL DROP LIKELY: Economists expect applications fell by 6,000 to a seasonally adjusted 333,000, according to a survey by FactSet. The Thanksgiving and Christmas holidays have warped recent benefits reports, causing sharp jumps and declines in recent weeks. Applications plunged by 39,000 the week before Christmas, after having climbed by 75,000 in the two weeks after Thanksgiving. Before the holiday volatility, unemployment benefit applications had ... (full story)
 November factory orders likely rose 1.8%, with soft nondurable orders alongside lower energy and commodity prices. Durable orders were previously reported up 3.5%, boosted by a gain in aircraft orders. The new information in the report will be on nondurable orders, which we forecast fell slightly in nominal value alongside lower energy and commodity prices. We forecast the November trade balance remained roughly stable at -USD40.4bn from -USD40.6bn in October. Energy prices moved lower in November, which is likely to bring down the nominal size of the import bill for crude oil products. We also look for softer ... (full story)
 The week of the new year saw the dollar and the yen make a comeback against the euro and the pound. And now, the real action returns: FOMC Meeting Minutes as well as important employment data culminating in the most important NFP event on Friday are joined by rate decisions in the euro-zone and in the UK, among other events. These are the main market movers on our list for the second week of 2014. Here is our outlook for the major events to influence forex trading. Last week US first weekly jobless claims report in 2014 posted a 339,000 increase in the number of new claims for unemployment benefits. The figure was ... (full story) Markets are already acting as if the bad jobs numbers released Friday may have been just noise, and that the economy is still doing fine. Already we're seeing interest rates rise again, and other markets continue to bet that the Fed will move toward tightening. Treasuries are falling and interests rates are rising this morning as investors sell bonds in the wake of the release of two better-than-expected economic data points: the New York Fed's Empire State Manufacturing Activity Index, and December producer prices. The Empire State index surged to 12.51 from December's 0.98 reading, and sub-components of the index ... (full story)
 ust about everyone (myself included) who ventured a payrolls forecast was crushed by the scant December gain of just 74k. How much should you adjust your outlook on the basis of just this one number? Not much, if at all. It is important to watch for trends in the data, and always keep Barry Ritholtz's warning in the back of your mind: ...we know from each month�s revisions that the initial read is off, often by a substantial amount. It�s a noisy series, subject to many errors and subsequent corrections. To put this into some context, consider what it is we are measuring: The change in monthly hires minus fires. A ... (full story)
 Friday�s disappointing and slightly perplexing jobs report is likely to curb the Federal Reserve�s recent enthusiasm about the U.S. economic recovery, but it seems unlikely to convince officials to alter the course Fed Chairman Ben Bernanke laid out for the central bank in December. That envisioned gradual reductions of its monthly bond-buying program this year. The Fed�s policy committee next meets January 28-29. The decision at hand will be whether to reduce the monthly bond buys from $75 billion in January to $65 billion until officials next meet in March. Mr. Bernanke strongly suggested at his December ... (full story)
 Curious why despite the huge miss in payrolls the unemployment rate tumbled from 7.0% to 6.7%? The reason is because in December the civilian labor force did what it usually does in the New Normal: it dropped from 155.3 million to 154.9 million, which means the labor participation rate just dropped to a fresh 35 year low, hitting levels not seen since 1978, at 62.8% down from 63.0%. And the piece de resistance: Americans not in the labor force exploded higher by 535,000 to a new all time high 91.8 million. The jobless, laborless recovery continues to steam on.
 Payrolls in December increased at the slowest pace since January 2011, indicating a pause in the recent strength of the U.S. labor market that may have reflected the effects of bad weather. The 74,000 gain in payrolls, less than the most pessimistic projection in a Bloomberg survey, followed a revised 241,000 advance the prior month, Labor Department figures showed today in Washington. The median forecast of 90 economists called for an increase of 197,000. The unemployment rate dropped to 6.7 percent, the lowest since October 2008, as more people left the labor force. More than a quarter million Americans were not at ... (full story)
 Household Survey Data The number of unemployed persons declined by 490,000 to 10.4 million in December, and the unemployment rate declined by 0.3 percentage point to 6.7 percent. Over the year, the number of unemployed persons and the unemployment rate were down by 1.9 million and 1.2 percentage points, respectively. (See table A-1.) Among the major worker groups, the unemployment rates for adult men (6.3 percent) and whites (5.9 percent) declined in December. The rates for adult women (6.0 percent), teenagers (20.2 percent), blacks (11.9 percent), and Hispanics (8.3 percent) showed little change. The jobless rate ... (full story)
 Finally it's here, the main event of the week, the release of US employment data. Is it to be the game changer that many are secretly anticipating? Is this the one that will finally burst through most people's expectations? It's the first employment report in awhile that many seem to have reserved their opinions, while quietly expecting the best � are we to be disappointed? This market is short the 18-member single currency both outright and on the crosses, and to be fair, probably a tad concerned after Draghi's press conference yesterday. Consensus expects the ECB to remain dovish, act dovish, but the timing of ... (full story)
 U.S. employment likely increased solidly in December in the latest sign of strengthening fundamentals that look set to put the economy on a faster growth path this year. Nonfarm payrolls probably rose by 196,000 jobs last month, according to a Reuters survey of economists, slightly below November's 203,000 gain, but a touch above the monthly average for the three months through November. The poll was conducted before reports on Wednesday, which showed private sector employers hired staff at the fastest pace in 13 months in December, while small businesses created the most jobs in nearly eight years. That left some ... (full story)
 The following are the expectations for the US December jobs report on Friday by the economists at Goldman Sachs, Bank of America Merrill Lynch, Citibank, Barclays Capital, and other major banks. GS: US Non-Farm Payroll Employment 175K, Unemployment Rate: 7.0%. BofA Merrill: The December employment report should be solid, with job growth of 220,000 and the unemployment rate holding at 7.0%. The data have turned decisively stronger at the end of the 2013, with GDP growth tracking 3.0% in 4Q after the strong 4.1% gain in 3Q. Consumer spending has accelerated and sentiment has improved, which is likely indicative of ... (full story)
 Employers probably expanded payrolls in December, capping the strongest year for U.S. employment since 2005, a report today may show. The addition of 197,000 jobs followed a 203,000 advance the prior month, according to the median forecast of 90 economists in a Bloomberg survey. The projected gain would bring the annual increase to 2.27 million, which was last exceeded eight years ago. The unemployment rate may have held at a five-year low of 7 percent in December. Ford Motor Co. (F:US) and Southwest Airlines Co. (LUV:US) are among companies taking on more staff as sales improve, which in turn will help sustain the ... (full story)
 Today�s most important piece of data is the US employment report. In addition to these three numbers, there will be plenty of other data releases today, from French monthly industrial production to India�s FX reserves, and also the Ifo Institute�s Eurozone economic outlook (15:00 GMT). With emerging markets suffering after the Federal Reserve�s decision to begin tapering, and Europe�s economic recovery being very fragmented, even normally minor data points could provide market-driving surprises, so do yourself a favour and note the daily releases. From the Fed, Jeffrey Lacker (13:45 GMT) and James Bullard ... (full story)
 US Non-Farm Employment Change measures the change in the number of newly employed people in the US, excluding workers in the farming industry. A reading which is higher than the market forecast is bullish for the dollar. Here are the details and 5 possible outcomes for EUR/USD. Published on Friday at 13:30 GMT. Indicator Background Job creation is one of the most important leading indicators of overall economic activity. The release of US Non-Farm Employment Change is highly anticipated by the markets, and an unexpected reading could affect the direction of EUR/USD. Despite concerns about dips in employment numbers ... (full story)
 At last the US economy seems to have rediscovered sustainable growth and that should be reflected in Friday's Non Farm Payroll numbers. A strong reading would support the US Federal Reserve's tapering narrative and boost USD. Q4 2013 saw a slew of very positive economic data out of the US with last month's NFP number showing that an extra 203,000 jobs were created. Also, previous NFP numbers were revised up. Therefore there is good reason to believe around 200,000 new posts were created during December. A poll of economist conducted by Thomson Reuters forecasts 195,000 new jobs with the unemployment rate at 7% though ... (full story)
 In the first full week of 2014, we identify four main sets of drivers for the foreign exchange market: the significance of last week's price action, interest rate differentials, central banks, and data releases. 1. Significance of last week's price action: Many observers have been discussing the dollar's weakness since the Fed announced its decision to begin slowing its long-term asset purchases. Instead, we have emphasized the divergent performance of the greenback, noting that its weakness was concentrated against small number of currencies that move with the euro and sterling's orbit. Against the yen, dollar-bloc ... (full story)
 November factory orders likely rose 1.8%, with soft nondurable orders alongside lower energy and commodity prices. Durable orders were previously reported up 3.5%, boosted by a gain in aircraft orders. The new information in the report will be on nondurable orders, which we forecast fell slightly in nominal value alongside lower energy and commodity prices. We forecast the November trade balance remained roughly stable at -USD40.4bn from -USD40.6bn in October. Energy prices moved lower in November, which is likely to bring down the nominal size of the import bill for crude oil products. We also look for softer ... (full story)
 The week of the new year saw the dollar and the yen make a comeback against the euro and the pound. And now, the real action returns: FOMC Meeting Minutes as well as important employment data culminating in the most important NFP event on Friday are joined by rate decisions in the euro-zone and in the UK, among other events. These are the main market movers on our list for the second week of 2014. Here is our outlook for the major events to influence forex trading. Last week US first weekly jobless claims report in 2014 posted a 339,000 increase in the number of new claims for unemployment benefits. The figure was ... (full story) Natural-gas futures on Thursday briefly pared their losses after the U.S. Energy Information Administration reported that supplies of natural gas fell 157 billion cubic feet for the week ended Jan. 3. The fall was a bit more than expected as analysts surveyed by Platts forecast a drop of between 148 billion cubic feet and 152 billion cubic feet. Total stocks now stand at 2.817 trillion cubic feet, down 528 billion cubic feet from a year ago and 315 billion cubic feet below the five-year average, the government said. February natural gas (NMN:NGG14) was at $4.16 per million British thermal units, down 6 cents, or ... (full story) Today�s most important piece of data is the US employment report. In addition to these three numbers, there will be plenty of other data releases today, from French monthly industrial production to India�s FX reserves, and also the Ifo Institute�s Eurozone economic outlook (15:00 GMT). With emerging markets suffering after the Federal Reserve�s decision to begin tapering, and Europe�s economic recovery being very fragmented, even normally minor data points could provide market-driving surprises, so do yourself a favour and note the daily releases. From the Fed, Jeffrey Lacker (13:45 GMT) and James Bullard ... (full story)
 There is probably little to expect from the first ECB meeting this year, at least in terms of conventional policy measures. President Draghi said in a recent interview that, based on the latest data, he saw �no immediate need to act� on policy rates, further emphasising the encouraging signs in financial markets and the �progress� made by Eurozone countries. Other Governing Council members made it clear that they were comfortable with the current monetary stance given the ongoing signs of a gradual recovery � the ECB�s central case. Inflation probably holds the key for further action, but downside risks ... (full story) <<
""".encode('utf-8', 'ignore').decode("utf-8", "ignore")

"\nBREAKING\n The Labor Department reports on the number of people who applied for U.S. unemployment benefits last week. The report will be released at 8:30 a.m. Eastern Thursday. SMALL DROP LIKELY: Economists expect applications fell by 6,000 to a seasonally adjusted 333,000, according to a survey by FactSet. The Thanksgiving and Christmas holidays have warped recent benefits reports, causing sharp jumps and declines in recent weeks. Applications plunged by 39,000 the week before Christmas, after having climbed by 75,000 in the two weeks after Thanksgiving. Before the holiday volatility, unemployment benefit applications had ... (full story)\n November factory orders likely rose 1.8%, with soft nondurable orders alongside lower energy and commodity prices. Durable orders were previously reported up 3.5%, boosted by a gain in aircraft orders. The new information in the report will be on nondurable orders, which we forecast fell slightly in nominal value alongside lower energy and commo

In [23]:
import re

def clean_text(text):
    # Remove unrecognizable characters (e.g., control characters, unusual symbols)
    text = re.sub(r'[^\x20-\x7E]+', ' ', text)  # Keep printable ASCII range
    # Replace multiple spaces or newlines with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove excess whitespaces
    text = text.strip()
    return text


In [24]:
clean_text("""
BREAKING
 The Labor Department reports on the number of people who applied for U.S. unemployment benefits last week. The report will be released at 8:30 a.m. Eastern Thursday. SMALL DROP LIKELY: Economists expect applications fell by 6,000 to a seasonally adjusted 333,000, according to a survey by FactSet. The Thanksgiving and Christmas holidays have warped recent benefits reports, causing sharp jumps and declines in recent weeks. Applications plunged by 39,000 the week before Christmas, after having climbed by 75,000 in the two weeks after Thanksgiving. Before the holiday volatility, unemployment benefit applications had ... (full story)
 November factory orders likely rose 1.8%, with soft nondurable orders alongside lower energy and commodity prices. Durable orders were previously reported up 3.5%, boosted by a gain in aircraft orders. The new information in the report will be on nondurable orders, which we forecast fell slightly in nominal value alongside lower energy and commodity prices. We forecast the November trade balance remained roughly stable at -USD40.4bn from -USD40.6bn in October. Energy prices moved lower in November, which is likely to bring down the nominal size of the import bill for crude oil products. We also look for softer ... (full story)
 The week of the new year saw the dollar and the yen make a comeback against the euro and the pound. And now, the real action returns: FOMC Meeting Minutes as well as important employment data culminating in the most important NFP event on Friday are joined by rate decisions in the euro-zone and in the UK, among other events. These are the main market movers on our list for the second week of 2014. Here is our outlook for the major events to influence forex trading. Last week US first weekly jobless claims report in 2014 posted a 339,000 increase in the number of new claims for unemployment benefits. The figure was ... (full story) Markets are already acting as if the bad jobs numbers released Friday may have been just noise, and that the economy is still doing fine. Already we're seeing interest rates rise again, and other markets continue to bet that the Fed will move toward tightening. Treasuries are falling and interests rates are rising this morning as investors sell bonds in the wake of the release of two better-than-expected economic data points: the New York Fed's Empire State Manufacturing Activity Index, and December producer prices. The Empire State index surged to 12.51 from December's 0.98 reading, and sub-components of the index ... (full story)
 ust about everyone (myself included) who ventured a payrolls forecast was crushed by the scant December gain of just 74k. How much should you adjust your outlook on the basis of just this one number? Not much, if at all. It is important to watch for trends in the data, and always keep Barry Ritholtz's warning in the back of your mind: ...we know from each month�s revisions that the initial read is off, often by a substantial amount. It�s a noisy series, subject to many errors and subsequent corrections. To put this into some context, consider what it is we are measuring: The change in monthly hires minus fires. A ... (full story)
 Friday�s disappointing and slightly perplexing jobs report is likely to curb the Federal Reserve�s recent enthusiasm about the U.S. economic recovery, but it seems unlikely to convince officials to alter the course Fed Chairman Ben Bernanke laid out for the central bank in December. That envisioned gradual reductions of its monthly bond-buying program this year. The Fed�s policy committee next meets January 28-29. The decision at hand will be whether to reduce the monthly bond buys from $75 billion in January to $65 billion until officials next meet in March. Mr. Bernanke strongly suggested at his December ... (full story)
 Curious why despite the huge miss in payrolls the unemployment rate tumbled from 7.0% to 6.7%? The reason is because in December the civilian labor force did what it usually does in the New Normal: it dropped from 155.3 million to 154.9 million, which means the labor participation rate just dropped to a fresh 35 year low, hitting levels not seen since 1978, at 62.8% down from 63.0%. And the piece de resistance: Americans not in the labor force exploded higher by 535,000 to a new all time high 91.8 million. The jobless, laborless recovery continues to steam on.
 Payrolls in December increased at the slowest pace since January 2011, indicating a pause in the recent strength of the U.S. labor market that may have reflected the effects of bad weather. The 74,000 gain in payrolls, less than the most pessimistic projection in a Bloomberg survey, followed a revised 241,000 advance the prior month, Labor Department figures showed today in Washington. The median forecast of 90 economists called for an increase of 197,000. The unemployment rate dropped to 6.7 percent, the lowest since October 2008, as more people left the labor force. More than a quarter million Americans were not at ... (full story)
 Household Survey Data The number of unemployed persons declined by 490,000 to 10.4 million in December, and the unemployment rate declined by 0.3 percentage point to 6.7 percent. Over the year, the number of unemployed persons and the unemployment rate were down by 1.9 million and 1.2 percentage points, respectively. (See table A-1.) Among the major worker groups, the unemployment rates for adult men (6.3 percent) and whites (5.9 percent) declined in December. The rates for adult women (6.0 percent), teenagers (20.2 percent), blacks (11.9 percent), and Hispanics (8.3 percent) showed little change. The jobless rate ... (full story)
 Finally it's here, the main event of the week, the release of US employment data. Is it to be the game changer that many are secretly anticipating? Is this the one that will finally burst through most people's expectations? It's the first employment report in awhile that many seem to have reserved their opinions, while quietly expecting the best � are we to be disappointed? This market is short the 18-member single currency both outright and on the crosses, and to be fair, probably a tad concerned after Draghi's press conference yesterday. Consensus expects the ECB to remain dovish, act dovish, but the timing of ... (full story)
 U.S. employment likely increased solidly in December in the latest sign of strengthening fundamentals that look set to put the economy on a faster growth path this year. Nonfarm payrolls probably rose by 196,000 jobs last month, according to a Reuters survey of economists, slightly below November's 203,000 gain, but a touch above the monthly average for the three months through November. The poll was conducted before reports on Wednesday, which showed private sector employers hired staff at the fastest pace in 13 months in December, while small businesses created the most jobs in nearly eight years. That left some ... (full story)
 The following are the expectations for the US December jobs report on Friday by the economists at Goldman Sachs, Bank of America Merrill Lynch, Citibank, Barclays Capital, and other major banks. GS: US Non-Farm Payroll Employment 175K, Unemployment Rate: 7.0%. BofA Merrill: The December employment report should be solid, with job growth of 220,000 and the unemployment rate holding at 7.0%. The data have turned decisively stronger at the end of the 2013, with GDP growth tracking 3.0% in 4Q after the strong 4.1% gain in 3Q. Consumer spending has accelerated and sentiment has improved, which is likely indicative of ... (full story)
 Employers probably expanded payrolls in December, capping the strongest year for U.S. employment since 2005, a report today may show. The addition of 197,000 jobs followed a 203,000 advance the prior month, according to the median forecast of 90 economists in a Bloomberg survey. The projected gain would bring the annual increase to 2.27 million, which was last exceeded eight years ago. The unemployment rate may have held at a five-year low of 7 percent in December. Ford Motor Co. (F:US) and Southwest Airlines Co. (LUV:US) are among companies taking on more staff as sales improve, which in turn will help sustain the ... (full story)
 Today�s most important piece of data is the US employment report. In addition to these three numbers, there will be plenty of other data releases today, from French monthly industrial production to India�s FX reserves, and also the Ifo Institute�s Eurozone economic outlook (15:00 GMT). With emerging markets suffering after the Federal Reserve�s decision to begin tapering, and Europe�s economic recovery being very fragmented, even normally minor data points could provide market-driving surprises, so do yourself a favour and note the daily releases. From the Fed, Jeffrey Lacker (13:45 GMT) and James Bullard ... (full story)
 US Non-Farm Employment Change measures the change in the number of newly employed people in the US, excluding workers in the farming industry. A reading which is higher than the market forecast is bullish for the dollar. Here are the details and 5 possible outcomes for EUR/USD. Published on Friday at 13:30 GMT. Indicator Background Job creation is one of the most important leading indicators of overall economic activity. The release of US Non-Farm Employment Change is highly anticipated by the markets, and an unexpected reading could affect the direction of EUR/USD. Despite concerns about dips in employment numbers ... (full story)
 At last the US economy seems to have rediscovered sustainable growth and that should be reflected in Friday's Non Farm Payroll numbers. A strong reading would support the US Federal Reserve's tapering narrative and boost USD. Q4 2013 saw a slew of very positive economic data out of the US with last month's NFP number showing that an extra 203,000 jobs were created. Also, previous NFP numbers were revised up. Therefore there is good reason to believe around 200,000 new posts were created during December. A poll of economist conducted by Thomson Reuters forecasts 195,000 new jobs with the unemployment rate at 7% though ... (full story)
 In the first full week of 2014, we identify four main sets of drivers for the foreign exchange market: the significance of last week's price action, interest rate differentials, central banks, and data releases. 1. Significance of last week's price action: Many observers have been discussing the dollar's weakness since the Fed announced its decision to begin slowing its long-term asset purchases. Instead, we have emphasized the divergent performance of the greenback, noting that its weakness was concentrated against small number of currencies that move with the euro and sterling's orbit. Against the yen, dollar-bloc ... (full story)
 November factory orders likely rose 1.8%, with soft nondurable orders alongside lower energy and commodity prices. Durable orders were previously reported up 3.5%, boosted by a gain in aircraft orders. The new information in the report will be on nondurable orders, which we forecast fell slightly in nominal value alongside lower energy and commodity prices. We forecast the November trade balance remained roughly stable at -USD40.4bn from -USD40.6bn in October. Energy prices moved lower in November, which is likely to bring down the nominal size of the import bill for crude oil products. We also look for softer ... (full story)
 The week of the new year saw the dollar and the yen make a comeback against the euro and the pound. And now, the real action returns: FOMC Meeting Minutes as well as important employment data culminating in the most important NFP event on Friday are joined by rate decisions in the euro-zone and in the UK, among other events. These are the main market movers on our list for the second week of 2014. Here is our outlook for the major events to influence forex trading. Last week US first weekly jobless claims report in 2014 posted a 339,000 increase in the number of new claims for unemployment benefits. The figure was ... (full story) Natural-gas futures on Thursday briefly pared their losses after the U.S. Energy Information Administration reported that supplies of natural gas fell 157 billion cubic feet for the week ended Jan. 3. The fall was a bit more than expected as analysts surveyed by Platts forecast a drop of between 148 billion cubic feet and 152 billion cubic feet. Total stocks now stand at 2.817 trillion cubic feet, down 528 billion cubic feet from a year ago and 315 billion cubic feet below the five-year average, the government said. February natural gas (NMN:NGG14) was at $4.16 per million British thermal units, down 6 cents, or ... (full story) Today�s most important piece of data is the US employment report. In addition to these three numbers, there will be plenty of other data releases today, from French monthly industrial production to India�s FX reserves, and also the Ifo Institute�s Eurozone economic outlook (15:00 GMT). With emerging markets suffering after the Federal Reserve�s decision to begin tapering, and Europe�s economic recovery being very fragmented, even normally minor data points could provide market-driving surprises, so do yourself a favour and note the daily releases. From the Fed, Jeffrey Lacker (13:45 GMT) and James Bullard ... (full story)
 There is probably little to expect from the first ECB meeting this year, at least in terms of conventional policy measures. President Draghi said in a recent interview that, based on the latest data, he saw �no immediate need to act� on policy rates, further emphasising the encouraging signs in financial markets and the �progress� made by Eurozone countries. Other Governing Council members made it clear that they were comfortable with the current monetary stance given the ongoing signs of a gradual recovery � the ECB�s central case. Inflation probably holds the key for further action, but downside risks ... (full story) <<

""")

"BREAKING The Labor Department reports on the number of people who applied for U.S. unemployment benefits last week. The report will be released at 8:30 a.m. Eastern Thursday. SMALL DROP LIKELY: Economists expect applications fell by 6,000 to a seasonally adjusted 333,000, according to a survey by FactSet. The Thanksgiving and Christmas holidays have warped recent benefits reports, causing sharp jumps and declines in recent weeks. Applications plunged by 39,000 the week before Christmas, after having climbed by 75,000 in the two weeks after Thanksgiving. Before the holiday volatility, unemployment benefit applications had ... (full story) November factory orders likely rose 1.8%, with soft nondurable orders alongside lower energy and commodity prices. Durable orders were previously reported up 3.5%, boosted by a gain in aircraft orders. The new information in the report will be on nondurable orders, which we forecast fell slightly in nominal value alongside lower energy and commodity p

In [25]:
len(prompt_template)

1733

In [30]:
sammm = """The FOMC meeting came and went with the expected result - the tapering process continued on schedule, undeterred by the current emerging market turmoil. Of course, the Fed doesn't want to be seen as reacting to every gyration financial markets. But even more importantly, the Fed wants out of the asset purchase business on the belief that a.) tapering is not tightening and b.) even if it was tightening, they could compensate via forward guidance. The global stumble, however, is challenging that thinking. Regardless of financial markets or US data, the Fed was not likely to launch into a new policy direction on the eve ... (full story)
 The voting members all supported the decision to taper an extra $10 billion a month in the last month that Federal Reserve Chairman Ben Bernanke presided. That�s the first time since June 2011 that a decision has been unanimous, with prior objections coming mostly due to hawks upset with the bond purchases in the first place, but last month due to one official fretting it was too soon to scale back. Never mind � the January decision brought four different regional Fed presidents � two of whom are ardent hawks (Richard Fisher and Charles Plosser), one who�s a converted dove (Narayana Kocherlakota), and one ... (full story)
 Federal Reserve officials stuck to their plan to reduce the central bank�s bond buying program to $65 billion per month. Here are five takes on what it means: VALIDATION OF THE TAPERING PLAN: Ben Bernanke suggested in December that the Federal Reserve would continue reducing the central bank�s monthly bond-buying in $10 billion increments at upcoming meetings, but he didn�t state it directly. He said the Fed would take �further modest steps� to reduce QE in the �general range� of $10 billion. Wednesday�s decision to pull back the bond-buying program to $65 billion per month is a validation of the ... (full story)
 The Federal Reserve will trim its monthly bond buying by $10 billion to $65 billion, sticking to its plan for a gradual withdrawal from departing Chairman Ben S. Bernanke�s unprecedented easing policy. �Labor market indicators were mixed but on balance showed further improvement,� the Federal Open Market Committee said today in a statement following a two-day meeting in Washington that was the last for Bernanke, who will be succeeded by Vice Chairman Janet Yellen on Feb. 1. �The unemployment rate declined but remains elevated.� Policy makers pressed on with a reduction in the purchases intended to speed a ... (full story)
 Information received since the Federal Open Market Committee met in December indicates that growth in economic activity picked up in recent quarters. Labor market indicators were mixed but on balance showed further improvement. The unemployment rate declined but remains elevated. Household spending and business fixed investment advanced more quickly in recent months, while the recovery in the housing sector slowed somewhat. Fiscal policy is restraining economic growth, although the extent of restraint is diminishing. Inflation has been running below the Committee's longer-run objective, but longer-term inflation ... (full story)
 Heading into this week�s top event risk, the Dow Jones FXCM Dollar Index (ticker = USDollar) has worked itself into a position that necessitates a breakout. That looks like a perfect marriage of build up and catalyst from the FOMC rate decision. Yet, despite its technical predicament and the ominous approach of a key catalyst, clearing this congestion does not necessarily mean there is a new trend in store for the benchmark currency. Engaging risk aversion after years of leveraged investment into record high asset prices is difficult to muster. However, all the elements are there to feed the fire should the initial ... (full story)
 The following are the expectations for today's FOMC January meeting by the economists at Goldman Sachs, Bank of America Merrill Lynch, Citibank, Barclays Capital and other major banks. GS: GS and consensus have Fed fund rate unchanged at 0.25% and expect further USD10bn reduction in the monthly rate of asset purchases to be announced � split equally between Treasuries and MBS, making them USD+35bn and USD+30bn, respectively. The January FOMC should be fairly uneventful, following significant policy changes made at the prior meeting. We expect no changes to the forward guidance. Although the 6.5% threshold mentioned ... (full story)
 As Federal Reserve Chairman Ben S. Bernanke shuts the door to his office for a final time in two days, he can say he took actions that were the first or the biggest of their kind in the central bank�s 100-year history. Some will probably also be the last. Bernanke was the first to devise a monetary policy that focused on lowering credit costs by suppressing longer-term interest rates after the short-term policy rate hit zero. His strategy, involving direct purchases of agency mortgage-backed securities and longer-term Treasury debt, left the Fed with the biggest balance sheet in its history, $4.1 trillion. He was ... (full story)
 Turmoil in emerging markets and a month of disappointing job growth at home are unlikely to deter the Federal Reserve from trimming its bond-buying stimulus on Wednesday, as Ben Bernanke wraps up his last policy meeting at the helm of the U.S. central bank. Overall signs of improvement in the U.S. economy suggest Fed officials will stay on track to cut monthly purchases of Treasuries and mortgage-backed securities by $5 billion each, bringing the total of their monthly asset purchases to $65 billion. The meeting is Bernanke's last before Vice Chair Janet Yellen moves into the top spot. Bernanke took the Fed far into ... (full story)
 Tomorrow, the Federal Open Market Committee announces its final monetary policy decision under the leadership of Federal Reserve chairman Ben Bernanke. At its December meeting, the FOMC finally took the plunge and began tapering down its quantitative easing (QE) program, reducing the pace of monthly purchases it makes in the U.S. Treasury and mortgage-backed securities (MBS) markets by $10 billion to $75 billion. The consensus forecast of Wall Street economists � per Bernanke's own comments at the conclusion of the December meeting � is that the FOMC will continue reducing bond purchases by $10 billion at each ... (full story)
 Ahead of the FOMC meeting on Wednesday February 29, speculations about possible alterations in the Fed's forward guidance are starting to gain some traction in the market and that has put the USD under some pressure as of late, notes Morgan Stanley. "Alongside speculation regarding adjustments to the pace of tapering, market expectations are likely to increase regarding the potential for the unemployment threshold to be adjusted from the current 6.5%, especially given that the latest US data have been far more mixed," MS clarifies. "However, our US economics team believes that the Fed is more likely to continue to ... (full story)
 Emerging market investors will watch the outcome of the Federal Reserve's policy meeting on Wednesday closely in hopes the central bank might throw them a bone, but analysts told CNBC they will most likely be disappointed. Emerging market currencies suffered an ugly selloff last week, after a sharp plunge in the Argentine peso, weak Chinese economic data and continued worries over further tapering by the Fed triggered a broader selloff across the region. But investors banking on a helping hand from the Fed could be left empty handed, analysts said. "It's pretty unlikely to be honest," said Robert ... (full story)
 Neither snow, freezing temperatures, market volatility nor a lousy jobs report will stay the Federal Reserve from taking another small step towards the exit this week. Federal Reserve officials will likely agree on another $10 billion taper to its bond-purchase program after a two-day meeting that ends Wednesday. That�s the same pace as the first reduction announced in December and it will bring the monthly purchases down to $65 billion per month, consisting of $35 billion of Treasurys and $30 billion or mortgage backed securities. �I think the Fed is desperate to extract itself from quantitative easing, and it ... (full story)
 Traders are bracing for another rough ride in markets this week, as the emerging markets continue to shakeout and the Fed is expected to announce another round of cuts to its easing program. There are also about a quarter of the S&P 500 companies reporting earnings, including Apple Monday, Boeing and Facebook Wednesday, and Exxon Mobil and Google Thursday. Several major economic reports are also expected, starting with new home sales Monday and durable goods Tuesday, while the first look at fourth-quarter GDP is expected Thursday. But the big event is the Fed meeting and how the markets will trade around that ... (full story)
 The U.S. Federal Reserve will take center stage in the week ahead with a widely expected cut to its bond-buying stimulus, responding to an improving U.S. economy but also helping fuel a dramatic emerging market sell-off. Argentina, Turkey and Ukraine felt the full force last week of a global flight from developing world assets as investors grew concerned about slower growth in China and U.S. monetary tightening, as well as the countries' own problems. Fed policymakers are expected to confirm the tightening trend during their Jan 28-29 meeting, notably also for being Ben Bernanke's last as chairman before vice chair ... (full story)
 The global capital markets imploded in the second half of last week and investors spent the weekend contemplating the significance. The main focus was on the emerging markets, which were especially hard hit. American-firsters (the tendency on the political right and left to blame the US first for all the undesirable things that happen in the world) jumped at the opportunity. Even in the Financial Times, which acknowledged other factors at work deemed worthy of a call out that the "Exit of money from Argentina has been driven by renewed concern about the US Fed". Really? That concern ostensibly is that the Fed is ... (full story)
 Plenty of data and Fedspeak to chew on last week, the sum total of which I think point in the same general direction. Economic activity is on average improving modestly, the Federal Reserve will push through with another round of tapering next week, and low inflation continues to hold back the threat of rate hikes. After stripping out the auto component, retail sales were solid in December: I think we are at or nearing the point where auto sales will generally move sideways and thus induce some additional volatility in the headline number. Consequently, it will be increasingly important to focus on core sales ... (full story)"""

In [35]:
sammm[:7000]

'The FOMC meeting came and went with the expected result - the tapering process continued on schedule, undeterred by the current emerging market turmoil. Of course, the Fed doesn\'t want to be seen as reacting to every gyration financial markets. But even more importantly, the Fed wants out of the asset purchase business on the belief that a.) tapering is not tightening and b.) even if it was tightening, they could compensate via forward guidance. The global stumble, however, is challenging that thinking. Regardless of financial markets or US data, the Fed was not likely to launch into a new policy direction on the eve ... (full story)\n The voting members all supported the decision to taper an extra $10 billion a month in the last month that Federal Reserve Chairman Ben Bernanke presided. That�s the first time since June 2011 that a decision has been unanimous, with prior objections coming mostly due to hawks upset with the bond purchases in the first place, but last month due to one 

In [15]:
too_long = 0
for index, row in parsed_data.iterrows():
    if row["sentiment_score"] > 1.0000:
        parsed_data.loc[index, "sentiment_score"] = np.float32(1.0000)
        too_long += 1
        print(index, row["link_text"])
print(too_long)

0


In [17]:
parsed_data['sentiment_score'].unique()

array([0.65  , 0.75  , 0.85  , 0.875 , 0.5   , 0.25  , 0.15  , 0.675 ,
       0.7   , 0.35  , 0.785 , 0.925 , 0.95  , 0.5423, 0.725 , 0.45  ,
       0.5432, 0.4   , 0.5489, 0.54  , 0.9   , 0.625 , 0.5875, 0.575 ,
       0.5417, 0.    , 0.7813, 0.485 , 0.2857, 0.475 , 0.6   , 0.545 ,
       0.4875, 0.0476, 0.085 , 0.425 , 0.05  , 0.5413, 0.5475, 0.9876,
       0.546 , 0.5429, 0.9875, 0.2   , 0.542 , 0.5473, 0.5478, 0.5476,
       0.535 , 0.685 , 0.1   , 0.5419, 0.5625, 0.6875, 0.125 , 0.765 ,
       0.281 , 0.5814, 0.3   , 0.7823, 0.525 , 0.55  , 0.075 , 1.    ,
       0.0375, 0.2345])

In [8]:
parsed_data.tail()

,time,open,high,low,close,tick_volume,spread,real_volume,impact,link_text,sentiment_score
270014,2024-11-15 22:45:00,1.05229,1.05280,1.05226,1.05266,849,16,0,Low Impact Expected,Good afternoon. Thank you for inviting me to s...,0.5
270015,2024-11-15 23:00:00,1.05267,1.05272,1.05215,1.05255,507,16,0,Low Impact Expected,Good afternoon. Thank you for inviting me to s...,0.5
270016,2024-11-15 23:15:00,1.05256,1.05288,1.05252,1.05284,513,26,0,Low Impact Expected,Good afternoon. Thank you for inviting me to s...,0.5
270017,2024-11-15 23:30:00,1.05283,1.05322,1.05279,1.05314,283,26,0,Low Impact Expected,Good afternoon. Thank you for inviting me to s...,0.5
270018,2024-11-15 23:45:00,1.05314,1.05414,1.05298,1.05369,709,26,0,Low Impact Expected,Good afternoon. Thank you for inviting me to s...,0.5


In [19]:
parsed_data.to_csv("../data/interim/parsed_scraped_data_clipped.csv", index=False)

In [18]:
np.float32(1.0000)

1.0